# Load data set

In [1]:
import os
from os import listdir
from os.path import isfile, join
docLabels = []
docLabels = [f for f in listdir("/Users/allen.wu/Desktop/doc2vec") if f.endswith('.json')]

In [2]:
print docLabels

['part000001.json']


In [3]:
import json

def read_json(json_file=''):
    data = json.loads(json_file)
    return data

In [14]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


def read_data(path, tokens_only=False):
    documents = []

    for fileName in docLabels:
        path = path+fileName

        with open(path) as f:
            i = 0
            for line in f.readlines():
                
                data = read_json(line)
                #print line
                #print data['title']
                preprocessd = gensim.utils.simple_preprocess(data['title'])
                '''
                if tokens_only:
                    yield gensim.utils.simple_preprocess(data['title'])
                else:
                    # For training data, add tags
                    print gensim.utils.simple_preprocess(data['title'])
                    yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(data['title']), [i])
                '''
                documents.append(TaggedDocument(preprocessd, [i]))
                i+=1
    return documents
                

In [15]:
documents = read_data('/Users/allen.wu/Desktop/doc2vec/', tokens_only=True)

In [20]:
train_corpus = list(documents)

In [21]:
train_corpus[:2]

[TaggedDocument(words=[u'the', u'best', u'volumizing', u'mascaras', u'for', u'fuller', u'and', u'longer', u'lashes'], tags=[0]),
 TaggedDocument(words=[u'process', u'system', u'isn', u'responsive', u'on', u'my', u'galaxy', u'android', u'forums', u'at', u'androidcentral', u'com'], tags=[1])]

In [25]:
model = gensim.models.doc2vec.Doc2Vec(min_count=2)
model.build_vocab(train_corpus)

In [28]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 1.33 s, sys: 586 ms, total: 1.92 s
Wall time: 1.3 s


225117

In [29]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

array([ 4.25582081e-02,  6.38986826e-02, -3.78522798e-02, -1.57792382e-02,
        1.67288706e-02, -8.66753701e-03, -3.79643310e-03, -3.52656506e-02,
        3.05140819e-02, -9.08636767e-03,  2.99244858e-02, -4.52669188e-02,
       -1.79966539e-02, -1.72422640e-03, -3.10162362e-03,  3.59312259e-02,
       -2.21189223e-02, -3.79572585e-02, -2.21228842e-02, -5.58566786e-02,
       -4.30665612e-02,  1.47093777e-02,  2.01751553e-02,  5.91613911e-02,
        1.56513825e-02,  3.46703641e-02,  3.20941140e-03,  5.30309835e-03,
       -4.18815203e-02, -5.89942839e-03, -4.25836071e-02,  1.09668560e-02,
        7.47179519e-03, -2.34596692e-02,  3.21635231e-02,  1.87294502e-02,
        7.01410370e-03, -1.27171539e-02,  1.00721866e-02, -4.58202660e-02,
       -1.90070756e-02,  3.26490887e-02, -1.90535132e-02, -4.67321090e-02,
        3.60926148e-03, -3.65707576e-02,  2.64394619e-02,  4.41454956e-03,
       -4.73735883e-04,  4.35933359e-02, -6.20516157e-03,  2.38285512e-02,
       -5.25492476e-04, -

In [30]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [32]:
import collections

collections.Counter(ranks)  # Results vary between runs due to random seeding and very small corpus

Counter({0: 3,
         1: 1,
         2: 1,
         3: 2,
         5: 4,
         6: 1,
         7: 2,
         8: 2,
         9: 1,
         10: 3,
         11: 1,
         13: 4,
         15: 1,
         16: 2,
         17: 3,
         18: 4,
         19: 1,
         20: 3,
         21: 1,
         22: 1,
         23: 1,
         26: 2,
         27: 3,
         28: 2,
         29: 2,
         32: 2,
         34: 2,
         35: 1,
         36: 1,
         37: 3,
         39: 1,
         41: 1,
         42: 1,
         43: 3,
         44: 1,
         45: 1,
         46: 3,
         48: 2,
         49: 1,
         50: 1,
         52: 2,
         53: 1,
         54: 2,
         55: 2,
         57: 2,
         58: 3,
         59: 4,
         60: 1,
         62: 1,
         63: 2,
         64: 1,
         65: 3,
         66: 2,
         67: 1,
         68: 2,
         69: 3,
         70: 1,
         71: 1,
         72: 1,
         73: 3,
         75: 2,
         76: 1,
         78: 1,
 

# Find most similar documents

In [33]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (4999): «sierra nevada snow boosts risk of large and destructive avalanches in lake tahoe area»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d100,n5,w5,mc2,s0.001,t3):

MOST (3668, 0.9820766448974609): «fbi hoops scandal recruiting woes personnel losses could put miami hurricanes in tough spot after march madness canes watch»

SECOND-MOST (2026, 0.9817002415657043): «what it like to fly in air canada business class executive pods complete with lie flat beds remote that lets you order food and lighting that claims to help with jet lag»

MEDIAN (505, 0.8870075941085815): «eastern brock worrell is nj com olympic conference wrestler of the year»

LEAST (3080, -0.7624403238296509): «the playground bully and the heedless mom»



# Generating doc2vec

In [41]:
f = open("/Users/allen.wu/Desktop/doc2vec/doc2vec.txt", "w")

for word, index in documents:
    print (word, index)
    vector = model.infer_vector(word)
    f.write(str(vector)+'\n')
    print '\n'
f.close()

([u'the', u'best', u'volumizing', u'mascaras', u'for', u'fuller', u'and', u'longer', u'lashes'], [0])


([u'process', u'system', u'isn', u'responsive', u'on', u'my', u'galaxy', u'android', u'forums', u'at', u'androidcentral', u'com'], [1])


([u'collective', u'action', u'is', u'unions', u'last', u'defense', u'and', u'high', u'court', u'on', u'verge', u'of', u'ending', u'it'], [2])


([u'redskins', u'qb', u'alex', u'smith', u'to', u'be', u'one', u'of', u'nfl', u'highest', u'paid', u'quarterbacks'], [3])


([u'treated', u'for', u'burns', u'at', u'nationwide', u'children', u'hospital', u'after', u'apartment', u'building', u'fire'], [4])


([u'woman', u'sues', u'former', u'mesa', u'officer', u'philip', u'brailsford', u'acquitted', u'of', u'murder'], [5])


([u'donald', u'trump', u'jr', u'cuddles', u'with', u'daughter', u'after', u'vanessa', u'files', u'for', u'divorce', u'xfinitymy', u'xfinity', u'com'], [6])


([u'one', u'person', u'injured', u'after', u'drive', u'by', u'shooting', u'in',

([u'insanely', u'affordable', u'destinations', u'for', u'spring'], [129])


([u'guide', u'to', u'grand', u'rapids', u'area', u'lenten', u'fish', u'fries', u'photos'], [130])


([u'emily', u'list', u'endorses', u'democrat', u'susan', u'wild', u'in', u'pa', u'congressional', u'race'], [131])


([u'putin', u'marches', u'on', u'toward', u'showdown', u'with', u'west'], [132])


([u'packers', u'trade', u'cb', u'damarious', u'randall', u'to', u'browns', u'get', u'back', u'qb', u'deshone', u'kizer'], [133])


([u'ex', u'massachusetts', u'gov', u'patrick', u'weighing', u'white', u'house', u'bid'], [134])


([u'flag', u'football', u'seminole', u'ridge', u'shuts', u'down', u'palm', u'beach', u'gardens'], [135])


([u'man', u'charged', u'after', u'gingerbread', u'treehouse', u'full', u'of', u'child', u'porn', u'found', u'in', u'woods'], [136])


([u'bill', u'maher', u'delivers', u'blistering', u'critique', u'of', u'democrats', u'political', u'courage'], [137])


([u'make', u'things', u'illegal', u

([u'what', u'lawson', u'schaffer', u'saw', u'in', u'around', u'the', u'back', u'pass', u'that', u'blew', u'up', u'his', u'phone'], [368])


([u'ben', u'carson', u'on', u'his', u'critics', u'people', u'are', u'so', u'stupid'], [369])


([u'exclusive', u'get', u'first', u'look', u'at', u'kennywood', u'thomas', u'town'], [370])


([u'woodbury', u'writes', u'its', u'own', u'history', u'captures', u'first', u'state', u'title'], [371])


([u'mikayla', u'pivec', u'leads', u'oregon', u'state', u'women', u'to', u'ncaa', u'tournament', u'upset', u'of', u'tennessee'], [372])


([u'std', u'test', u'filed', u'in', u'jim', u'carrey', u'lawsuit', u'confirms', u'positive', u'results'], [373])


([u'the', u'will', u'mcdonough', u'writing', u'contest', u'winners', u'honored'], [374])


([u'power', u'lunch', u'chat', u'live', u'with', u'columnist', u'berry', u'tramel'], [375])


([u'things', u'to', u'know', u'about', u'shopping', u'at', u'warehouse', u'clubs'], [376])


([u'from', u'manning', u'rambo', u

([u'techies', u'are', u'dieting', u'by', u'loading', u'up', u'on', u'meat', u'and', u'veggies', u'for', u'full', u'month'], [665])


([u'britt', u'robertson', u'harsh', u'words', u'for', u'arie', u'luyendyk', u'jr'], [666])


([u'today', u'gun', u'control', u'debate', u'has', u'echoes', u'of'], [667])


([u'city', u'of', u'springfield', u'and', u'red', u'cross', u'providing', u'refuge', u'for', u'victims', u'of', u'tragic', u'belmont', u'avenue', u'fire'], [668])


([u'new', u'castle', u'county', u'police', u'recruits', u'shoot', u'weapons', u'for', u'first', u'time'], [669])


([u'man', u'shot', u'in', u'the', u'arm', u'in', u'northeast', u'baltimore', u'wednesday', u'night', u'police', u'say'], [670])


([u'things', u'women', u'should', u'know', u'about', u'taxes', u'after', u'divorce'], [671])


([u'psychological', u'tricks', u'restaurants', u'use', u'to', u'make', u'you', u'spend', u'more', u'money'], [672])


([u'tom', u'benson', u'restored', u'hope', u'to', u'his', u'hometown', u


([u'trevor', u'siemian', u'trade', u'to', u'minnesota', u'vikings', u'being', u'finalized'], [1121])


([u'junior', u'lineman', u'trey', u'derouen', u'no', u'longer', u'on', u'south', u'carolina', u'roster'], [1122])


([u'drive', u'for', u'uber', u'deliver', u'for', u'amazon', u'here', u'are', u'tax', u'rules', u'to', u'know'], [1123])


([u'greg', u'cromer', u'is', u'the', u'best', u'choice', u'for', u'slidell', u'mayor', u'editorial'], [1124])


([u'recruiting', u'trail', u'mcentire', u'johnson', u'add', u'offers'], [1125])


([u'ji', u'man', u'choi', u'continuing', u'to', u'make', u'an', u'impact', u'in', u'brewers', u'camp'], [1126])


([u'grad', u'transfer', u'qb', u'decommits', u'from', u'alabama', u'headed', u'to', u'pac'], [1127])


([u'after', u'the', u'las', u'vegas', u'shooting', u'massacre', u'survival', u'can', u'be', u'excruciating'], [1128])


([u'brexit', u'bulletin', u'ireland', u'again'], [1129])


([u'korbin', u'williams', u'central', u'catholic', u'wr', u'commits'

...

([u'guide', u'to', u'michigan', u'ncaa', u'tournament', u'site', u'wichita', u'intrust', u'bank', u'arena'], [3237])


([u'alarm', u'blaze', u'in', u'san', u'francisco', u'spreads', u'to', u'nearby', u'buildings'], [3238])


([u'pressure', u'cooker', u'mini', u'teriyaki', u'turkey', u'sandwiches'], [3239])


([u'ou', u'football', u'sam', u'bradford', u'on', u'josh', u'heupel', u'deflategate', u'and', u'his', u'future', u'with', u'the', u'st', u'louis', u'rams'], [3240])


([u'chesapeake', u'home', u'engulfed', u'in', u'flames'], [3241])


([u'storms', u'strike', u'college', u'leave', u'trail', u'of', u'damage', u'across', u'south'], [3242])


([u'hpd', u'man', u'shot', u'after', u'allegedly', u'sending', u'threatening', u'messages', u'to', u'another', u'male'], [3243])


([u'one', u'of', u'the', u'last', u'sweet', u'corn', u'growers', u'on', u'the', u'front', u'range', u'is', u'ending', u'that', u'harvest'], [3244])


([u'this', u'year', u'most', u'popular', u'travel', u'destinations']

([u'construction', u'woes', u'coming', u'to', u'route', u'at', u'easton', u'phillipsburg', u'line'], [3656])


([u'portland', u'racist', u'past', u'smolders', u'beneath', u'the', u'surface'], [3657])


([u'some', u'grosse', u'pointe', u'students', u'sit', u'out', u'protest', u'claiming', u'opinion', u'suppression'], [3658])


([u'hines', u'ward', u'troy', u'polamalu', u'and', u'others', u'will', u'be', u'involved', u'in', u'charlie', u'ebersol', u'new', u'football', u'league'], [3659])


([u'what', u'does', u'second', u'cousin', u'once', u'removed', u'mean', u'anyway', u'reader', u'digest'], [3660])


([u'oakland', u'raiders', u'trade', u'jamize', u'olawale', u'to', u'the', u'dallas', u'cowboys'], [3661])


([u'photo', u'editing', u'apps', u'that', u'will', u'make', u'you', u'never', u'trust', u'anything', u'you', u'see', u'on', u'instagram', u'again'], [3662])


([u'st', u'louis', u'art', u'museum', u'to', u'debut', u'ancient', u'sculptures', u'from', u'egypt'], [3663])


([u'engineer

([u'west', u'virginia', u'vs', u'marshall', u'won', u'happen', u'during', u'season', u'but', u'it', u'on', u'for', u'ncaa', u'tournament'], [4039])


([u'the', u'new', u'york', u'times', u'has', u'wildly', u'swinging', u'forecast', u'needle', u'that', u'driving', u'everyone', u'insane'], [4040])


([u'da', u'trooper', u'violated', u'training', u'agency', u'rulebook', u'when', u'frisking', u'woman', u'without', u'cause'], [4041])


([u'citi', u'exec', u'shares', u'the', u'one', u'critical', u'trait', u'that', u'made', u'her', u'successful', u'and', u'it', u'something', u'everyone', u'can', u'develop'], [4042])


([u'kentucky', u'republican', u'leader', u'says', u'pension', u'deal', u'will', u'be', u'difficult', u'to', u'pass'], [4043])


([u'after', u'cancer', u'st', u'mark', u'grad', u'phillips', u'returns', u'to', u'baseball', u'field'], [4044])


([u'how', u'did', u'stephen', u'hawking', u'live', u'so', u'long', u'with', u'als'], [4045])


([u'missing', u'pearl', u'river', u'resident



([u'cirque', u'du', u'soleil', u'performer', u'dies', u'after', u'fall', u'during', u'tampa', u'show', u'volta', u'canceled'], [4403])


([u'judge', u'just', u'ruled', u'on', u'whether', u'the', u'million', u'powerball', u'winner', u'can', u'keep', u'her', u'name', u'secret'], [4404])


([u'the', u'death', u'of', u'dslrs', u'is', u'near'], [4405])


([u'rick', u'ross', u'not', u'on', u'life', u'support', u'family', u'and', u'friends', u'say'], [4406])


([u'portland', u'elementary', u'students', u'honor', u'unity', u'on', u'walk', u'out', u'day'], [4407])


([u'report', u'this', u'is', u'what', u'the', u'central', u'banks', u'are', u'really', u'afraid', u'of', u'episode', u'politics'], [4408])


([u'are', u'you', u'ready', u'to', u'talk', u'about', u'the', u'master', u'of', u'none', u'ending'], [4409])


([u'star', u'wars', u'the', u'last', u'jedi', u'supposedly', u'tells', u'us', u'who', u'rey', u'parents', u'are', u'here', u'why', u'we', u're', u'not', u'buying', u'it'], [4410])






([u'nfl', u'free', u'agency', u'donald', u'stephenson', u'to', u'sign', u'with', u'browns'], [4759])


([u'peta', u'bashes', u'idaho', u'teacher', u'for', u'allegedly', u'feeding', u'puppy', u'to', u'snapping', u'turtle'], [4760])


([u'the', u'bachelor', u'bekah', u'reveals', u'arie', u'dm', u'her', u'after', u'she', u'left', u'the', u'show'], [4761])


([u'russia', u'says', u'it', u'stopped', u'mitt', u'romney', u'from', u'becoming', u'secretary', u'of', u'state', u'report'], [4762])


([u'man', u'told', u'nurses', u'will', u'kill', u'you', u'convicted', u'of', u'five', u'counts', u'of', u'terroristic', u'threats'], [4763])


([u'halliburton', u'campus', u'in', u'houston', u'westchase', u'district', u'to', u'be', u'auctioned', u'off'], [4764])


([u'are', u'your', u'measuring', u'spoons', u'lying', u'to', u'you'], [4765])


([u'how', u'to', u'root', u'broken', u'branch', u'on', u'christmas', u'cactus', u'hunker'], [4766])


([u'nfl', u'rumors', u'giants', u'ask', u'dominique', u'ro